Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/mouse-biosamples/snyder_2months_multiomics_B6XCast_July22_2021.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [4]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [5]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,description,Strain:skip,"Age, Sex, ID:skip",sex_symbol:skip,biosample_ontology,biosample_term_name:skip,date_obtained:date,Birth Date:skip,...,donor,source,lab,award,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,NaN,NaN,Encode B6CastF1 2 months female #1 Adrenal Glands,Encode B6CastF1,2 months ♀ #1,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-02,2021-04-05,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Encode B6CastF1 2 months female #2 Adrenal Glands,Encode B6CastF1,2 months ♀ #2,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-02,2021-04-05,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Encode B6CastF1 2 months female #3 Adrenal Glands,Encode B6CastF1,2 months ♀ #3,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-02,2021-04-05,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Encode B6CastF1 2 months female #4 Adrenal Glands,Encode B6CastF1,2 months ♀ #4,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-02,2021-04-05,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Encode B6CastF1 2 months female #5 Adrenal Glands,Encode B6CastF1,2 months ♀ #5,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-09,2021-04-18,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,Encode B6CastF1 2 months female #6 Adrenal Glands,Encode B6CastF1,2 months ♀ #6,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-09,2021-04-18,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Encode B6CastF1 2 months female #7 Adrenal Glands,Encode B6CastF1,2 months ♀ #7,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-23,2021-04-30,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,Encode B6CastF1 2 months female #8 Adrenal Glands,Encode B6CastF1,2 months ♀ #8,♀,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-23,2021-04-30,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Encode B6CastF1 2 months male #1 Adrenal Glands,Encode B6CastF1,2 months ♂ #1,♂,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-03,2021-04-05,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,Encode B6CastF1 2 months male #2 Adrenal Glands,Encode B6CastF1,2 months ♂ #2,♂,/biosample-types/tissue_UBERON_0002369/,Adrenal Glands,2021-06-03,2021-04-05,...,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [7]:
#biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

#for i, row in biosample.iterrows():
#    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
#        obj = server.get_json(row.accession)
#        biosample_ontology = obj['biosample_ontology']
#        if isinstance(biosample_ontology, dict):
#            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
#            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
#        biosample.loc[i, 'source'] = obj['source']['@id']
#            
#biosample

In [10]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=False, 
                            validator=validator)
print(len(created))

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS430ZGT', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:13.374759+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0002369/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months female #1 Adrenal Glands', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-02', 'starting_amount': 14, 'starting_amount_units': 'μg', 'model_organism_sex': 'female', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS430ZGT/', '@type': ['Biosample', 'Item'], 'uuid': '8d947923-0702-4133-8a0

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS227YDW', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:15.402502+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0002369/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months female #7 Adrenal Glands', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-23', 'starting_amount': 14, 'starting_amount_units': 'μg', 'model_organism_sex': 'female', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS227YDW/', '@type': ['Biosample', 'Item'], 'uuid': 'd71a20a0-e34d-4af0-b85

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS119TRC', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:17.813218+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0002369/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months male #5 Adrenal Glands', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-03', 'starting_amount': 14, 'starting_amount_units': 'μg', 'model_organism_sex': 'male', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS119TRC/', '@type': ['Biosample', 'Item'], 'uuid': '62299fea-6e7d-4e52-81c8-f7

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS618FPS', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:19.811347+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0001388/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months male #1 Gastroc', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-03', 'model_organism_sex': 'male', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS618FPS/', '@type': ['Biosample', 'Item'], 'uuid': '47776fac-c932-412e-8c94-0d1468fe233a', 'sex': 'male', 'age': '2', 'age_units': 'month',

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS905JZC', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:21.787287+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0002305/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months female #1 Hippocampus', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-02', 'starting_amount': 24, 'starting_amount_units': 'μg', 'model_organism_sex': 'female', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS905JZC/', '@type': ['Biosample', 'Item'], 'uuid': '6b6a403f-ff29-4b5e-a755-1

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS281GRU', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:23.804439+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0002305/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months male #1 Hippocampus', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-03', 'starting_amount': 24, 'starting_amount_units': 'μg', 'model_organism_sex': 'male', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS281GRU/', '@type': ['Biosample', 'Item'], 'uuid': '99420aa9-7ff2-48a2-9734-3a4aa

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS680HBI', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-07-26T23:46:25.741824+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0000956/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'Encode B6CastF1 2 months female #1 L. Cortex', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2021-06-02', 'starting_amount': 50, 'starting_amount_units': 'μg', 'model_organism_sex': 'female', 'model_organism_age': '2', 'model_organism_age_units': 'month', '@id': '/biosamples/ENCBS680HBI/', '@type': ['Biosample', 'Item'], 'uuid': '56f01bfc-7cb7-4775-a8f8-f78

In [11]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)